In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
import csv
import time


model_path='./model/model.ckpt'

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.' \
                                                            '  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # Define the name of the tensors
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'

    # Get the needed layers' outputs for building FCN-VGG16
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    image_input = tf.get_default_graph().get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = tf.get_default_graph().get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3 = tf.get_default_graph().get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4 = tf.get_default_graph().get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7 = tf.get_default_graph().get_tensor_by_name(vgg_layer7_out_tensor_name)

    return image_input, keep_prob, layer3, layer4, layer7



def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """


    weights_regularized_l2 = 1e-4

    # 1x1 Convolution to preserve spatial information.
    enc_layer7 = tf.layers.conv2d(
        vgg_layer7_out, num_classes, kernel_size=1,
        kernel_initializer= tf.random_normal_initializer(stddev=0.01),
        kernel_regularizer= tf.contrib.layers.l2_regularizer(weights_regularized_l2), name='enc_layer7')

    enc_layer4 = tf.layers.conv2d(
        vgg_layer4_out, num_classes, kernel_size=1,
        kernel_initializer= tf.random_normal_initializer(stddev=0.01),
        kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-4), name='enc_layer4')
    enc_layer3 = tf.layers.conv2d(
        vgg_layer3_out, num_classes, kernel_size=1,
        kernel_initializer= tf.random_normal_initializer(stddev=0.01),
        kernel_regularizer= tf.contrib.layers.l2_regularizer(weights_regularized_l2), name='enc_layer3')

    # Deconvolution:  Upsample
    dec_layer1 = tf.layers.conv2d_transpose(
        enc_layer7, num_classes, kernel_size=4, strides=(2, 2),
        padding='same',
        kernel_initializer= tf.random_normal_initializer(stddev=0.01),
        kernel_regularizer= tf.contrib.layers.l2_regularizer(weights_regularized_l2), name='dec_layer1')

    # Skip connection from the vgg_layer4_out
    dec_layer2 = tf.add(
        dec_layer1, enc_layer4, name='dec_layer2')

    # Deconvolution: Make shape the same as layer3
    dec_layer3 = tf.layers.conv2d_transpose(
        dec_layer2, num_classes, kernel_size=4, strides=(2, 2),
        padding='same',
        kernel_initializer= tf.random_normal_initializer(stddev=0.01),
        kernel_regularizer= tf.contrib.layers.l2_regularizer(weights_regularized_l2), name='dec_layer3')

    # Same for layer4.
    dec_layer4 = tf.add(
        dec_layer3, enc_layer3, name='dec_layer4')
    decoder_output = tf.layers.conv2d_transpose(
        dec_layer4, num_classes, kernel_size=16, strides=(8, 8),
        padding='same',
        kernel_initializer= tf.random_normal_initializer(stddev=0.01),
        kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-4), name='dec_layer4')

    return decoder_output







def predict_images(test_data_path, print_speed=False):
    num_classes = 2
    image_shape = (160, 576)
    runs_dir = './runs'

    # Path to vgg model
    vgg_path = os.path.join('./data', 'vgg')

    with tf.Session() as sess:
        # Predict the logits
        input_image, keep_prob, vgg_layer3_out, vgg_layer4_out, vgg_layer7_out = load_vgg(sess, vgg_path)
        nn_last_layer = layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes)
        logits = tf.reshape(nn_last_layer, (-1, num_classes))

        # Restore the saved model
        saver = tf.train.Saver()
        saver.restore(sess, model_path)
        print("Load saved Model in file: %s" % model_path)

        # Predict the samples
        helper.pred_samples(runs_dir, test_data_path, sess, image_shape, logits, keep_prob, input_image, print_speed)






/home/stack/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow Version: 1.6.0


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: UserWarning: No GPU found. Please use a GPU to train your neural network.


In [2]:

print ('############    PREDICT IMAGES')
test_data_path = './data/data_road/testing/image_2'

predict_images(test_data_path, print_speed=True)


############    PREDICT IMAGES
INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
INFO:tensorflow:Restoring parameters from ./model/model.ckpt
Load saved Model in file: ./model/model.ckpt
################    Predicting images
Processing file: 00001,	Speed: 0.32 fps
Processing file: 00002,	Speed: 0.33 fps
Processing file: 00003,	Speed: 0.33 fps
Processing file: 00004,	Speed: 0.33 fps
Processing file: 00005,	Speed: 0.33 fps
Processing file: 00006,	Speed: 0.33 fps
Processing file: 00007,	Speed: 0.33 fps
Processing file: 00008,	Speed: 0.33 fps
Processing file: 00009,	Speed: 0.33 fps
Processing file: 00010,	Speed: 0.33 fps
Processing file: 00011,	Speed: 0.33 fps
Processing file: 00012,	Speed: 0.33 fps
Processing file: 00013,	Speed: 0.33 fps
Processing file: 00014,	Speed: 0.33 fps
Processing file: 00015,	Speed: 0.33 fps
Processing file: 00016,	Speed: 0.34 fps
Processing file: 00017,	Speed: 0.33 fps
Processing file: 00018,	Speed: 0.32 fps
Processing file: 00019,	Speed

Processing file: 00200,	Speed: 0.32 fps
Processing file: 00201,	Speed: 0.33 fps
Processing file: 00202,	Speed: 0.31 fps
Processing file: 00203,	Speed: 0.33 fps
Processing file: 00204,	Speed: 0.32 fps
Processing file: 00205,	Speed: 0.33 fps
Processing file: 00206,	Speed: 0.33 fps
Processing file: 00207,	Speed: 0.32 fps
Processing file: 00208,	Speed: 0.32 fps
Processing file: 00209,	Speed: 0.32 fps
Processing file: 00210,	Speed: 0.30 fps
Processing file: 00211,	Speed: 0.29 fps
Processing file: 00212,	Speed: 0.33 fps
Processing file: 00213,	Speed: 0.31 fps
Processing file: 00214,	Speed: 0.32 fps
Processing file: 00215,	Speed: 0.34 fps
Processing file: 00216,	Speed: 0.31 fps
Processing file: 00217,	Speed: 0.32 fps
Processing file: 00218,	Speed: 0.29 fps
Processing file: 00219,	Speed: 0.30 fps
Processing file: 00220,	Speed: 0.32 fps
Processing file: 00221,	Speed: 0.33 fps
Processing file: 00222,	Speed: 0.25 fps
Processing file: 00223,	Speed: 0.32 fps
Processing file: 00224,	Speed: 0.29 fps
